In [1]:
!pip3 install Jetson.GPIO

  Preparing metadata (setup.py) ... done
  Created wheel for Jetson.GPIO: filename=Jetson.GPIO-2.1.6-py3-none-any.whl size=31259 sha256=7af060b4861e7be453a0524a34907f8832d3dd0fdc19f03d016429a2ee547d27
  Stored in directory: /Users/ta/Library/Caches/pip/wheels/35/d8/6f/090f0cee346695c0db3f117f3e62906c7bbe0da7d4aee545f8
Successfully built Jetson.GPIO


In [2]:
import sys
sys.path.append('../train')

import traitlets
import ipywidgets.widgets as widgets
from ipywidgets import HTML
from IPython.display import display

import numpy as np
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import time


from learn import SimpleCNN,SimpleCNN_1,CustomResNet,CustomCircleDataset
# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg
import Jetson.GPIO as GPIO

ModuleNotFoundError: No module named 'jetbot'

In [ ]:
camera = Camera()

widget_width = 224
widget_height = 224

camera_widget = widgets.Image(format='jpg', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpg', width=widget_width, height=widget_height)
image_layout = widgets.HBox([camera_widget, target_widget])

#mask slider
low_h_slider = widgets.IntSlider(description='low h', min=0, max=179, value=90,step=1)
high_h_slider = widgets.IntSlider(description=' high h', min=0, max=179, value=100,step=1)
low_s_slider = widgets.IntSlider(description=' low s', min=0, max=255, value=140,step=1)
high_s_slider = widgets.IntSlider(description=' high s', min=0, max=255, value=255,step=1)
low_v_slider = widgets.IntSlider(description=' low v', min=0, max=255, value=0,step=1)
high_v_slider = widgets.IntSlider(description=' high v', min=0, max=255, value=255,step=1)

h_slider = widgets.HBox([low_h_slider, high_h_slider])
s_slider = widgets.HBox([low_s_slider, high_s_slider])
v_slider = widgets.HBox([low_v_slider, high_v_slider])
slider = widgets.VBox([h_slider,s_slider,v_slider])

#robot param slider
far_slider = widgets.IntSlider(description='far rad', min=0, max=100, value=20,step=1)
near_slider = widgets.IntSlider(description=' near rad', min=0, max=100, value=30,step=1)
speed_slider = widgets.FloatSlider(description='speed', min=0, max=1.0, value=0,step=0.1)
interval_slider = widgets.FloatSlider(description=' interval', min=0, max=5.0, value=0.5,step=0.1)
robot_slider = widgets.VBox([far_slider, near_slider, speed_slider, interval_slider])

layout = widgets.Layout(width='128px', height='64px')
snap_button = widgets.Button(description='snapshot', button_style='success', layout=layout)
percent = widgets.IntText(layout = layout,value = 0)
gui_layout =  widgets.HBox([image_layout,percent])

In [ ]:
# デバイスの設定（CUDAが利用可能な場合はGPUを使用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルのインスタンスを作成
model = SimpleCNN()
# モデルの重みをロードし、推論モードに設定
model.load_state_dict(torch.load('weight/model_2.pth'))
model.to(device)  # モデルをGPUに移動
model.eval()

In [ ]:
pos_x, pos_y, pos_r = 0,0,0

def apply_hsv_threshold(image):
#     low_h, high_h = 91, 103  # Example range for yellow hue
#     low_s, high_s = 200, 255 # Example range for saturation
#     low_v, high_v = 140, 255 # Example range for value
    low_h, high_h = low_h_slider.value, high_h_slider.value
    low_s, high_s = low_s_slider.value, high_s_slider.value
    low_v, high_v = low_v_slider.value, high_v_slider.value

    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define the lower and upper bounds of the HSV threshold
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])
    
    # Create a mask where pixels within the threshold are white, and others are black
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Create an all black image
    black_image = np.zeros_like(image)
    
    # Copy the pixels from the original image where the mask is white
    result_image = np.where(mask[:, :, None] == 255, image, black_image)
    
    return result_image

def calibration(image):
    # カメラ行列と歪み係数を設定（キャリブレーションプロセスから取得）
    camera_matrix = np.array([[108.16614089 ,  0 ,        111.55104361],
                            [  0,         143.70743912 ,121.87531702],
                            [  0,           0,           1,        ]])
    distortion_coefficients = np.array([-0.33140309 , 0.12007599, -0.00293275,  0.00091844 ,-0.02038985])

    # 補正したい画像を読み込む
    undistorted_image = cv2.undistort(image, camera_matrix, distortion_coefficients)
    
    return undistorted_image

def display_xy(camera_image):
    image = np.copy(camera_image)
    # image = calibration(image)
    image = apply_hsv_threshold(image)
    
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    image = transform(pil_image).unsqueeze(0).to(device)  # テンソルをGPUに移動

    # モデルを使用して予測
    global pos_x, pos_y, pos_r
    with torch.no_grad():
        outputs = model(image)
        predicted_coords = outputs.cpu().numpy()[0]  # 結果をCPUに戻す
        pos_x, pos_y, pos_r = predicted_coords

    pos_x = int(pos_x * camera_image.shape[1])
    pos_y = int(pos_y * camera_image.shape[0])
    pos_r = int(pos_r * (camera_image.shape[1] * 1.414))
    if pos_r < 0:
        pos_r = 0
        
    

    img = np.copy(camera_image)
    img = apply_hsv_threshold(img)
    
    img = cv2.circle(img, (pos_x, pos_y), pos_r, (0, 255, 0), 2)
    jpeg_image = bgr8_to_jpeg(img)
    return jpeg_image

traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

In [ ]:
left_layout = widgets.VBox([image_layout,slider])
right_layout = widgets.VBox([robot_slider])
layout_ui = widgets.HBox([left_layout, right_layout])
display(layout_ui)

In [ ]:
GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
GPIO.setup(31,GPIO.IN) #トグルスイッチ左
GPIO.setup(33,GPIO.OUT) #トグルスイッチ中，ボタンHIGH側
GPIO.setup(35,GPIO.IN) #トグルスイッチ右
GPIO.setup(37,GPIO.IN) #ボタンLOW側

GPIO.output(33,GPIO.HIGH) #HIGH出力

move = [] #[[左,右],...]
robot = Robot()
def sigmoid(x, k=0.06, x0=112):
    if x==0:
        return 0
    else:
        return 2 / (1 + np.exp(-k * (x - x0)))

while True:

    button = GPIO.input(35) #実行ボタンの入力を受け取る

    toggle0 = GPIO.input(31) #トグルスイッチ左向き（記録モード）
    toggle1 = GPIO.input(35) #トグルスイッチ右向き（実行モード）

    if toggle0 == GPIO.HIGH:#記録モード
        global pos_x,pos_r
        x = pos_x

        v = x - 112 #まんなか，左寄りなら負，右寄りなら正
        rad  = pos_r
        scale = sigmoid(abs(v)) #vの影響力を調整する係数

        speed = speed_slider.value #基本スピード

        interval = interval_slider.value #間隔
        
        
        
        #はやさの判定
        if rad<far_slider.value:
            speed = 0.5
        elif far_slider.value<rad and rad<near_slider.value:
            speed = 0.2
        elif near_slider.value<rad:
            speed = 0
        

        left_speed = speed + v*speed*scale #左モーター出力
        right_speed = speed - v*speed*scale #右モーター出力
        
        robot.set_motors(left_speed,right_speed) #左右のモーター出力
        move.append([left_speed,right_speed]) #配列に記録(後ろに追加)
        time.sleep(interval)
        if toggle0 == GPIO.LOW:
            break
        
    elif toggle1 == GPIO.HIGH:#実行モード
        #初期状態を記録
        exe = 0
        before = button
        now = button
        if exe ==0:#実行待機
            before = now #前の状態
            now = button #現在の状態
            if before == GPIO.LOW and now == GPIO.HIGH: #LOW→HIGH（押した瞬間）を検知
                exe = 1
    
        elif exe == 1:#実行部分
            for i in move:
                robot.set_motors(move[i][0],move[i][1])
                time.sleep(interval)
            exe = 0